In [4]:
import pandas as pd
import numpy as np
from pgmpy.models import BayesianModel
from pgmpy.estimators import ConstraintBasedEstimator
from scipy import stats

In [5]:
# loading data
raw = pd.read_csv("forestfires.csv")
data = pd.DataFrame(raw)
data.head()

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0


In [ ]:
est = ConstraintBasedEstimator(data)
skel, sep_sets = est.estimate_skeleton()

/home/sven/anaconda3/lib/python3.6/site-packages/pgmpy/estimators/base.py:202: UserWarning: Insufficient data for testing X _|_ FFMC | []. At least 4200.0 samples recommended, 517 present.
  "At least {0} samples recommended, {1} present.".format(5 * num_params, len(self.data)))
/home/sven/anaconda3/lib/python3.6/site-packages/pgmpy/estimators/base.py:202: UserWarning: Insufficient data for testing X _|_ DMC | []. At least 8560.0 samples recommended, 517 present.
  "At least {0} samples recommended, {1} present.".format(5 * num_params, len(self.data)))
/home/sven/anaconda3/lib/python3.6/site-packages/pgmpy/estimators/base.py:202: UserWarning: Insufficient data for testing X _|_ DC | []. At least 8720.0 samples recommended, 517 present.
  "At least {0} samples recommended, {1} present.".format(5 * num_params, len(self.data)))
/home/sven/anaconda3/lib/python3.6/site-packages/pgmpy/estimators/base.py:202: UserWarning: Insufficient data for testing X _|_ ISI | []. At least 4720.0 samples r

/home/sven/anaconda3/lib/python3.6/site-packages/pgmpy/estimators/base.py:202: UserWarning: Insufficient data for testing day _|_ area | []. At least 7500.0 samples recommended, 517 present.
  "At least {0} samples recommended, {1} present.".format(5 * num_params, len(self.data)))
/home/sven/anaconda3/lib/python3.6/site-packages/pgmpy/estimators/base.py:202: UserWarning: Insufficient data for testing FFMC _|_ month | []. At least 5775.0 samples recommended, 517 present.
  "At least {0} samples recommended, {1} present.".format(5 * num_params, len(self.data)))
/home/sven/anaconda3/lib/python3.6/site-packages/pgmpy/estimators/base.py:202: UserWarning: Insufficient data for testing FFMC _|_ day | []. At least 3150.0 samples recommended, 517 present.
  "At least {0} samples recommended, {1} present.".format(5 * num_params, len(self.data)))
/home/sven/anaconda3/lib/python3.6/site-packages/pgmpy/estimators/base.py:202: UserWarning: Insufficient data for testing FFMC _|_ DMC | []. At least 11

/home/sven/anaconda3/lib/python3.6/site-packages/pgmpy/estimators/base.py:202: UserWarning: Insufficient data for testing DC _|_ rain | []. At least 6540.0 samples recommended, 517 present.
  "At least {0} samples recommended, {1} present.".format(5 * num_params, len(self.data)))
/home/sven/anaconda3/lib/python3.6/site-packages/pgmpy/estimators/base.py:202: UserWarning: Insufficient data for testing DC _|_ area | []. At least 272500.0 samples recommended, 517 present.
  "At least {0} samples recommended, {1} present.".format(5 * num_params, len(self.data)))
/home/sven/anaconda3/lib/python3.6/site-packages/pgmpy/estimators/base.py:202: UserWarning: Insufficient data for testing ISI _|_ month | []. At least 6490.0 samples recommended, 517 present.
  "At least {0} samples recommended, {1} present.".format(5 * num_params, len(self.data)))
/home/sven/anaconda3/lib/python3.6/site-packages/pgmpy/estimators/base.py:202: UserWarning: Insufficient data for testing ISI _|_ day | []. At least 3540

/home/sven/anaconda3/lib/python3.6/site-packages/pgmpy/estimators/base.py:202: UserWarning: Insufficient data for testing wind _|_ month | []. At least 1100.0 samples recommended, 517 present.
  "At least {0} samples recommended, {1} present.".format(5 * num_params, len(self.data)))
/home/sven/anaconda3/lib/python3.6/site-packages/pgmpy/estimators/base.py:202: UserWarning: Insufficient data for testing wind _|_ DMC | []. At least 21400.0 samples recommended, 517 present.
  "At least {0} samples recommended, {1} present.".format(5 * num_params, len(self.data)))
/home/sven/anaconda3/lib/python3.6/site-packages/pgmpy/estimators/base.py:202: UserWarning: Insufficient data for testing wind _|_ DC | []. At least 21800.0 samples recommended, 517 present.
  "At least {0} samples recommended, {1} present.".format(5 * num_params, len(self.data)))
/home/sven/anaconda3/lib/python3.6/site-packages/pgmpy/estimators/base.py:202: UserWarning: Insufficient data for testing wind _|_ ISI | []. At least 1

In [ ]:
skel.edges()

In [ ]:
pdag = est.skeleton_to_pdag(skel, sep_sets)
pdag.edges()